# Intro notebook
The purpose of this notebook is to connect to a local database that we created by loading CSV files into that database.
Copy / paste this notebook into other places to add / delete cells and add new analysis.

In [1]:
import os
print(os.getcwd())

/Users/beoconno/Documents/ff/ff_data/notebooks/nfl


In [2]:
from pathlib import Path

In [18]:
# Define your database file path (SQLite example)
rel_db_path = Path.cwd().parent.parent / 'raw_data' / 'nfl' / 'src_code' / 'nfl.db'
db_file_path = rel_db_path.resolve()

In [19]:
print(db_file_path)

/Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db


In [20]:
engine = create_engine(f'sqlite:///{db_file_path}')
print(engine)

Engine(sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db)


In [21]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [22]:
%sql sqlite:///{db_file_path}

In [23]:
%%sql
SELECT *
FROM nfl_results
LIMIT 10

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


ranker,player,team,fantasy_pos,age,g,gs,pass_cmp,pass_att,pass_yds,pass_td,pass_int,rush_att,rush_yds,rush_yds_per_att,rush_td,targets,rec,rec_yds,rec_yds_per_rec,rec_td,fumbles,fumbles_lost,all_td,two_pt_md,two_pt_pass,fantasy_points,fantasy_points_ppr,draftkings_points,fanduel_points,vbd,fantasy_rank_pos,fantasy_rank_overall,player_id,Year
1,David Johnson*+,ARI,RB,25,16,16,0,0,0,0,0,293,1239,4.23,16,120,80,879,10.99,4,5.0,3,20,1.0,None,328.0,407.8,416.8,367.8,191.0,1,1.0,JohnDa08,2016
2,Ezekiel Elliott*+,DAL,RB,21,15,15,0,0,0,0,0,322,1631,5.07,15,39,32,363,11.34,1,5.0,1,16,None,None,293.0,325.4,332.4,309.4,156.0,2,2.0,ElliEz00,2016
3,Aaron Rodgers*,GNB,QB,33,16,16,401,610,4428,40,7,67,369,5.51,4,0,0,0,None,0,8.0,4,4,1.0,1.0,380.0,380.0,397.0,387.0,119.0,1,3.0,RodgAa00,2016
4,LeSean McCoy*,BUF,RB,28,15,15,0,0,0,0,0,234,1267,5.41,13,57,50,356,7.12,1,3.0,0,14,1.0,None,248.0,298.3,304.3,273.3,111.0,3,4.0,McCoLe01,2016
5,Le'Veon Bell*,PIT,RB,24,12,12,0,1,0,0,0,261,1268,4.86,7,94,75,616,8.21,2,4.0,1,9,1.0,None,242.0,317.4,324.4,279.9,105.0,4,5.0,BellLe00,2016
6,DeMarco Murray*,TEN,RB,28,16,16,1,2,10,1,0,293,1287,4.39,9,67,53,377,7.11,3,3.0,1,12,None,None,241.0,293.8,300.8,267.3,104.0,5,6.0,MurrDe00,2016
7,Devonta Freeman*,ATL,RB,24,16,16,0,0,0,0,0,227,1079,4.75,11,65,54,462,8.56,2,1.0,1,13,None,None,230.0,284.1,291.1,257.1,93.0,6,7.0,FreeDe00,2016
8,LeGarrette Blount,NWE,RB,30,16,8,0,0,0,0,0,299,1161,3.88,18,8,7,38,5.43,0,2.0,1,18,None,None,226.0,232.9,236.9,229.4,89.0,7,8.0,BlouLe00,2016
9,Matt Ryan*+,ATL,QB,31,16,16,373,534,4944,38,7,35,117,3.34,0,0,0,0,None,0,4.0,2,0,1.0,1.0,347.0,347.5,362.5,354.5,87.0,2,9.0,RyanMa00,2016
10,Mike Evans*,TAM,WR,23,16,16,0,0,0,0,0,0,0,None,0,173,96,1321,13.76,12,0.0,0,12,2.0,None,208.0,304.1,307.1,256.1,82.0,1,10.0,EvanMi00,2016


In [24]:
%%sql
SELECT
    year,
    fantasy_pos,
    count(1) AS player_cnt
FROM nfl_results
GROUP BY 1, 2
ORDER BY 1, 2

 * sqlite:////Users/beoconno/Documents/ff/ff_data/raw_data/nfl/src_code/nfl.db
Done.


Year,fantasy_pos,player_cnt
2016,FB,5
2016,QB,73
2016,RB,163
2016,TE,124
2016,WR,226
2017,FB,15
2017,QB,75
2017,RB,156
2017,TE,127
2017,WR,227
